In [48]:
from bs4 import BeautifulSoup as BS
import pymorphy2
import requests
import re
import csv
from nltk import sent_tokenize, word_tokenize, pos_tag


verbs = []
nouns = []
f = open('C:/Users/asans/Downloads/Verbs.txt', encoding='utf-8')
for line in f:
    verbs.append(line)
f = open('word_rus.txt', encoding='utf-8')
for line in f:
    nouns.append(line[:-1])
    
Nouns = []
Verbs = []
for noun in nouns:
    nn = noun.replace('ё','е')
    nn2 = nn.replace('-','')
    Nouns.append(nn2.lower())
for v in verbs:
    Verbs.append(v.lower()[:-1])
Verbs.remove('быть')
Verbs.remove('делать')

In [49]:
nouns_var = open('Nouns.txt','w')
for noun in Nouns:
    nouns_var.write(noun+'\n')
nouns_var.close()

In [50]:
verbs_var = open('Verbs.txt','w')
for verb in Verbs:
    verbs_var.write(verb+'\n')
verbs_var.close()

In [51]:
def to_norm(w):
    morph = pymorphy2.MorphAnalyzer()
    p = morph.parse(w)[0]
    return p.normal_form

In [52]:
import spacy
nlp = spacy.load("ru_core_news_sm")

def to_norm(token):
    # Используем spaCy для получения нормальной формы слова
    return token.lemma_

def Pairs(appeal):
    sent_list = sent_tokenize(appeal, language='russian')
    P_list = []
    
    for sent in sent_list:
        words_list = word_tokenize(sent, language='russian')
        WL = [re.sub('[^а-яё]', '', wli.lower()) for wli in words_list if len(wli) > 1]
        WL = list(filter(('').__ne__, WL))
        
        doc = nlp(' '.join(WL))
        
        f1, f2 = 0, 0
        first_part, second_part = None, None
        
        for token in doc:
            if len(token.text) > 1:
                tp = to_norm(token)
                if token.pos_ == 'NOUN' and tp in Nouns:
                    first_part = tp
                    f1 += 1
                if token.pos_ == 'VERB' and tp in Verbs:
                    second_part = tp
                    f2 += 1
            
            if f1 > 0 and f2 > 0:
                P_list.append((first_part, second_part))
                f1, f2 = 0, 0
                first_part, second_part = None, None
    
    return P_list


In [53]:
appeals = []
with open('training.csv', 'r', encoding='utf-8') as tr:
    train_sample = csv.reader(tr, delimiter=';')
    for row in train_sample:
        try:
            n, h = int(row[0]), int(row[3])
        except:
            continue
        pr = Pairs(row[2])
        print(n,pr,h)
        if len(pr)>1:
            appeals.append((Pairs(row[2]),h,n))

1 [] 2
2 [('знак', 'находиться'), ('причина', 'стать'), ('знак', 'ставить')] 2
3 [('день', 'тратить'), ('филиал', 'располагаться'), ('суббота', 'представляться'), ('филиал', 'работать')] 1
4 [('массаж', 'спрашивать'), ('спортсмен', 'знать'), ('жизнь', 'жаловаться'), ('праздник', 'помогать')] 1
5 [('зарплата', 'иметься'), ('ипотека', 'давать')] 1
6 [] 5
7 [('декабрь', 'находиться'), ('человек', 'видеть')] 5
8 [('опыт', 'видеть')] 4
9 [('техника', 'производить'), ('вопрос', 'возникать'), ('газон', 'восстанавливать')] 2
10 [('человек', 'обходить'), ('грязь', 'вступать'), ('школа', 'ходить')] 2
11 [('дом', 'жить'), ('инвалид', 'пройти'), ('бок', 'ездить')] 2
12 [('вопрос', 'обращаться'), ('учитель', 'работать'), ('дорога', 'переходить'), ('переход', 'существовать'), ('год', 'сбивать'), ('запрос', 'осуществляться')] 2
13 [('популяризация', 'служить'), ('человек', 'идти'), ('водитель', 'ехать'), ('рубль', 'покупать'), ('пакет', 'надевать'), ('рука', 'резать'), ('человек', 'менять'), ('здоров

In [ ]:
with open('appeals.csv', 'w', newline='', encoding='utf-8') as w:
    writer = csv.writer(w)
    for ap in appeals:
        writer.writerow(ap)
len(appeals)

77

In [55]:
counts = [0,0,0,0,0]
for ap in appeals:
    counts[ap[1]-1]+=1
print(counts)

[25, 24, 9, 6, 13]
